# 📘 Para-Procesimi i të Dhënave për Analizë
## Tema: Analiza e të dhënave të shitjeve (Superstore Dataset)

Ky notebook përmban hapat e detajuar të para-procesimit për dataset-in **Superstore**.
Procesi përfshin:
- Ngarkimin dhe inspektimin e të dhënave
- Trajtimin e tipeve të dhënave dhe vlerave të zbrazëta
- Pastrimin, agregimin dhe mostrimin
- Diskretizimin, binarizimin dhe transformimet
- Reduktimin e dimensioneve me PCA

**👩‍🎓 Studentja:** Anjeza Sfishta  <br>
**📚 Lënda:** Përgatitja dhe vizualizimi i të dhënave  

## 🧩 Importimi i librarive
Në këtë seksion importojmë libraritë kryesore që do të përdorim gjatë para-procesimit.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, Binarizer, KBinsDiscretizer
from sklearn.decomposition import PCA

print('Pandas version:', pd.__version__)
print('Seaborn version:', sns.__version__)

## 📥 Ngarkimi i të dhënave
Lexojmë dataset-in dhe shikojmë rreshtat e parë për të kuptuar strukturën e tij.


In [ ]:
df = pd.read_csv('superstore.csv')
df.head()

## 🔍 Inspektimi dhe cilësia e të dhënave
Analizojmë tipet, mungesat dhe statistikat për të vlerësuar cilësinë e dataset-it.


In [ ]:
df.info()
df.describe()
df.isnull().sum()

## 🧹 Pastrimi i të dhënave
Heqim duplikatet dhe rreshtat me vlera të zbrazëta, si dhe konvertojmë kolonat e datës në formatin e duhur.


In [ ]:
df = df.drop_duplicates()
#df = df.dropna()
for col in ['Sales']:
    df[col].fillna(df[col].median(), inplace=True)
    
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce')

invalid_dates = df[df['Order Date'].isna() | df['Ship Date'].isna()]
print(f"Rreshta me data të pavlefshme: {len(invalid_dates)}")

print("✅ Të dhënat u pastruan dhe datat u konvertuan me sukses!")

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Vizualizimi i vlerave të zbrazëta')
plt.show()

df[['Sales']].hist(bins=30, figsize=(6,4))
plt.title('Shpërndarja e Sales')
plt.show()

## ⚙️ Diskretizimi, Binarizimi dhe Standardizimi
Kryejmë:
- **Diskretizimin** e kolonës `Sales` në 4 intervale  
- **Binarizimin** e kolonës `Profit` për të ndarë fitimet nga humbjet  
- **Standardizimin** e kolonave numerike për t’i sjellë në një shkallë të përbashkët


In [ ]:
disc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
df['Sales_binned'] = disc.fit_transform(df[['Sales']])


scaler = StandardScaler()
df[['Row ID', 'Postal Code', 'Sales']] = scaler.fit_transform(df[['Row ID', 'Postal Code', 'Sales']])

## 🔗 Correlation Matrix për kolonat numerike


In [ ]:
# Gjej kolonat numerike automatikisht
num_cols = df.select_dtypes(include=['number']).columns

plt.figure(figsize=(6,5))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


## 🎯 Reduktimi i dimensioneve me PCA
Përdorim **PCA (Principal Component Analysis)** për të reduktuar dimensionet e të dhënave dhe për të vizualizuar strukturën e tyre.


In [ ]:
pca = PCA(n_components=1)
pca_data = pca.fit_transform(df[['Sales']])
df['PCA1'] = pca_data[:, 0]

plt.figure(figsize=(10,6))
sns.histplot(df['PCA1'], kde=True, color='skyblue', alpha=0.6)
plt.title('Vizualizimi i komponentit të parë PCA (Sales)')
plt.xlabel('PCA1')
plt.ylabel('Shpërndarja')
plt.show()

explained_variance = pca.explained_variance_ratio_[0]
print(f"Varianca e kapur nga PCA1: {explained_variance:.2f}")

## 🧠 Krijimi i veçorive të reja
Shtohet veçoria e re `Delivery_Days` që tregon kohën e dërgesës për çdo porosi.


In [ ]:
df['Delivery_Days'] = (df['Ship Date'] - df['Order Date']).dt.days
df[['Order Date', 'Ship Date', 'Delivery_Days']].head()

## 📊 Mostrim dhe Agregim i të Dhënave

Në këtë seksion do të kryejmë disa analiza përmbledhëse duke përdorur funksione të tilla si:
- `groupby()` për të grupuar të dhënat sipas kategorive ose rajoneve
- `agg()` për të llogaritur mesatare, total dhe numërime
- `sample()` për të marrë mostra të rastësishme nga dataset-i

Këto analiza ndihmojnë për të kuptuar shpërndarjen e shitjeve dhe fitimeve sipas rajonit, kategorisë ose segmentit të klientit.


🧩 Shembuj Praktikë

1. Mostrim i rastësishëm (5 rreshta)

In [ ]:
print("Mostër e rastësishme nga dataset-i:")
display(df.sample(5))


2. Mesatarja e shitjeve dhe fitimit sipas Rajonit

In [ ]:
print("\nMesatarja e shitjeve dhe fitimit sipas Rajonit:")
region_summary = df.groupby('Region')[['Sales']].mean().reset_index()
display(region_summary)

3. Totali i shitjeve dhe fitimit sipas Kategorisë së Produktit

In [ ]:
print("\nTotali i shitjeve dhe fitimit sipas Kategorisë së Produktit:")
category_summary = df.groupby('Category')[['Sales',]].sum().reset_index()
display(category_summary)

4. Numri i porosive sipas Segmentit të Klientit

In [ ]:
print("\nNumri i porosive sipas Segmentit të Klientit:")
segment_orders = df['Segment'].value_counts().reset_index()
segment_orders.columns = ['Segment', 'Numri i Porosive']
display(segment_orders)

5. Vizualizim grafik i shitjeve mesatare sipas Rajonit

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=region_summary, x='Region', y='Sales')
plt.title("💰 Mesatarja e Shitjeve sipas Rajonit", fontsize=13)
plt.xlabel("Rajoni")
plt.ylabel("Mesatarja e Shitjeve")
plt.show()

## 🏁 Përmbledhje përfundimtare
Këtu përmblidhen të gjithë hapat e realizuar gjatë procesit të para-procesimit.  
Dataset-i përfundimtar u ruajt për përdorim të mëtejshëm në analizë apo modelim.


In [ ]:
print("✅ Para-procesimi u përfundua me sukses!\n")
print("U kryen këto hapa:")
print("1. Ngarkimi dhe inspektimi i të dhënave")
print("2. Pastrimi dhe konvertimi i tipeve")
print("3. Diskretizimi dhe binarizimi")
print("4. Standardizimi dhe PCA")
print("5. Krijimi i veçorive të reja")

df.to_csv("cleaned_superstore.csv", index=False)
print("\n💾 Të dhënat e pastruara u ruajtën si 'cleaned_superstore.csv'")
